In [1]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

rng = np.random.default_rng(42)
t = np.arange(0, 120)
trend = 0.05 * t
season = 2.0 * np.sin(2 * np.pi * t / 12)
noise = rng.normal(0, 0.5, size=t.size)
y = trend + season + noise

y_diff = np.diff(y, n=1)

fig = make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.1)
fig.add_trace(go.Scatter(x=t, y=y, name="original"), row=1, col=1)
fig.add_trace(go.Scatter(x=t[1:], y=y_diff, name="first difference"), row=2, col=1)
fig.update_layout(height=500, title="Original vs First Difference")
fig.show()

In [2]:
import plotly.graph_objects as go

from sktime.datasets import load_airline
from sktime.forecasting.compose import TransformedTargetForecaster
from sktime.forecasting.model_selection import ForecastingHorizon, temporal_train_test_split
from sktime.forecasting.naive import NaiveForecaster
from sktime.transformations.series.boxcox import BoxCoxTransformer
from sktime.transformations.series.detrend import Detrender, Deseasonalizer

y = load_airline()
y_train, y_test = temporal_train_test_split(y, test_size=36)
fh = ForecastingHorizon(y_test.index, is_relative=False)

pipe = TransformedTargetForecaster(steps=[
    ("boxcox", BoxCoxTransformer()),
    ("deseasonalize", Deseasonalizer(model="additive", sp=12)),
    ("detrend", Detrender()),
    ("forecaster", NaiveForecaster(strategy="drift")),
])

pipe.fit(y_train)
pred = pipe.predict(fh)

fig = go.Figure()
fig.add_trace(go.Scatter(x=y_train.index, y=y_train, name="train"))
fig.add_trace(go.Scatter(x=y_test.index, y=y_test, name="test"))
fig.add_trace(go.Scatter(x=pred.index, y=pred, name="forecast"))
fig.update_layout(title="TransformedTargetForecaster pipeline")
fig.show()

ModuleNotFoundError: No module named 'sktime'

In [3]:
import pandas as pd

df = pd.DataFrame({"y": y})
df["lag_1"] = df["y"].shift(1)
df["lag_12"] = df["y"].shift(12)
df["roll_mean_12"] = df["y"].rolling(12).mean()
df.head(15)

,y,lag_1,lag_12,roll_mean_12
0,0.152359,NaN,NaN,NaN
1,0.530008,0.152359,NaN,NaN
2,2.207276,0.530008,NaN,NaN
3,2.620282,2.207276,NaN,NaN
4,0.956533,2.620282,NaN,NaN
5,0.598910,0.956533,NaN,NaN
6,0.363920,0.598910,NaN,NaN
7,-0.808121,0.363920,NaN,NaN
8,-1.340451,-0.808121,NaN,NaN
9,-1.976522,-1.340451,NaN,NaN
